In [2]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 \
    pandas --quiet

%pip install \
    transformers==4.27.2 \
    datasets==2.11.0 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.3.0 --quiet


import pandas as pd
from datasets import Dataset  # Import the Dataset class
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, TrainingArguments, Trainer
import torch
import time

from datasets import Dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 80.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 82.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.
torchtext 0.16.0 re

In [3]:
# Load data from Excel
excel_path = 'NIAA RGS _ GIM changes_Edited.xlsx'
df = pd.read_excel(excel_path)

# Assuming your Excel columns are named 'Topic' and 'Message'
dataset = Dataset.from_pandas(df)

In [4]:
model_name = 'google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [8]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

#Test the Model with Zero Shot Inferencing

index = 28

dialogue = dataset['dialogue'][index]
summary = dataset['summary '][index]

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')


trainable model parameters: 247577856
all model parameters: 247577856
percentage of trainable model parameters: 100.00%
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following conversation.

Scholarships

Summary:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
GIM offers scholarship to the students based on their merit and financial background.                                Goa Institute of Management offers scholarship worth Rs. 5 lakhs each to the meritorious students so that money doesn't become a barrier in learning. Click below to know more.                                        https://gim.ac.in/admission/fees-and-financial-aid

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
Obtain a scholarship.


In [21]:
#Perform Full Fine-Tuning

def tokenize_function(example):
    start_prompt = 'Summarize the following conversation.\n\n'
    end_prompt = '\n\nSummary: '

    # Check for None values in the 'dialogue' column
    dialogues = example["dialogue"]
    prompt = [start_prompt + (dialogue if dialogue is not None else '') + end_prompt for dialogue in dialogues]

    # Ensure the column name matches the one in your dataset
    summaries = example["summary "]

    # Initialize empty lists to store tokenized input and labels
    input_ids_list = []
    labels_list = []

    for single_prompt, single_summary in zip(prompt, summaries):
        # Tokenize each example individually
        tokenized_output = tokenizer(single_prompt, single_summary, padding="max_length", truncation=True, return_tensors="pt", max_length=512)

        # Append tokenized input and labels to the lists
        input_ids_list.append(tokenized_output['input_ids'])
        labels_list.append(tokenized_output['input_ids'])  # Use 'input_ids' as labels for summarization

    # Convert lists to PyTorch tensors
    example['input_ids'] = torch.cat(input_ids_list, dim=0)
    example['labels'] = torch.cat(labels_list, dim=0)

    return example


# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['dialogue', 'summary '])

tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)

print(f"Shapes of the datasets:")
#print(f"Training: {tokenized_datasets['train'].shape}")
#print(f"Validation: {tokenized_datasets['validation'].shape}")
#print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Map:   0%|          | 0/64 [00:00<?, ? examples/s]

Filter:   0%|          | 0/64 [00:00<?, ? examples/s]

Shapes of the datasets:
Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 1
})


In [22]:
# Fine-Tune the Model with the Preprocessed Dataset

output_dir = f'./dialogue-summary-training-{str(int(time.time()))}'

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=1,
    max_steps=1
)

trainer = Trainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_datasets,  # Use the entire preprocessed dataset
    eval_dataset=tokenized_datasets  # Use the entire preprocessed dataset for evaluation
)

# Fine-tune the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
1,43.500000


TrainOutput(global_step=1, training_loss=43.5, metrics={'train_runtime': 3.7766, 'train_samples_per_second': 2.118, 'train_steps_per_second': 0.265, 'total_flos': 684757352448.0, 'train_loss': 43.5, 'epoch': 1.0})